# 一句话定义
一个用来标准化 应用 如何 向 LLM 提供工具和context 的开源协议

MCP 包含三个 核心的 组件：
- MCP Host: The AI application that coordinates and manages one or multiple MCP clients
- MCP Client: A component that maintains a connection to an MCP server and obtains context from an MCP server for the MCP host to use
- MCP Server: A program that provides context to MCP clients

# 在LangChain(以下简称LC) 使用MCP
Host 就是 用来管理 Clients的，在LC框架中：
````py
    from langchain_mcp_adapters.client import MultiServerMCPClient  
````

定义 Client，多个Client定义在一起：
````py
    client = MultiServerMCPClient(  
        {
            "math": {
                "transport": "stdio",  # Local subprocess communication
                "command": "python",
                # Absolute path to your math_server.py file
                "args": ["/path/to/math_server.py"],
            },
            "weather": {
                "transport": "streamable_http",  # HTTP-based remote server
                # Ensure you start your weather server on port 8000
                "url": "http://localhost:8000/mcp",
            }
        }
    )
````

Server 还是用 MCP SDK 去写：        
可以参看官方文档 [Build an MCP server](https://modelcontextprotocol.io/docs/develop/build-server)
````py
    from mcp.server.fastmcp import FastMCP

    mcp = FastMCP("Weather")

    @mcp.tool()
    async def get_weather(location: str) -> str:
        """Get weather for location."""
        return "It's always sunny in New York"

    if __name__ == "__main__":
        mcp.run(transport="streamable-http")
````

| 维度 | LC 原生 @Tool | MCP |
|-----|-------------|-----|
| **学习曲线** | 低 | 中等 |
| **开发速度** | 很快 | 中等 |
| **代码复杂度** | 低 | 中等 |
| **适合工具数量** | < 10 个 | > 10 个 |
| **服务独立性** | ❌ 紧耦合 | ✅ 松耦合 |
| **独立部署** | ❌ 不支持 | ✅ 支持 |
| **多应用共享** | ❌ 困难 | ✅ 容易 |
| **维护成本** | 中等 | 低（规模大时） |
| **实时更新** | ❌ 需要重启 | ✅ 无需重启 |
| **工具复用** | ❌ 不易 | ✅ 容易 |